## print the url for each WARC file

In [4]:
from warcio.archiveiterator import ArchiveIterator
from urllib.parse import urlparse
with open('common_crawl_bbc_sample.warc', 'rb') as stream:
    for record in ArchiveIterator(stream):
        if record.rec_type == 'response':
            url = record.rec_headers.get_header('WARC-Target-URI')
            url_split = urlparse(url)
            print(url_split)
            print(record.rec_headers.get_header('WARC-Target-URI'))

ParseResult(scheme='http', netloc='news.bbc.co.uk', path='/2/hi/africa/3414345.stm', params='', query='', fragment='')
http://news.bbc.co.uk/2/hi/africa/3414345.stm


## WARC Writing

In [1]:
from warcio.capture_http import capture_http
from warcio import WARCWriter
import requests  # requests *must* be imported after capture_http

with open('example.warc.gz', 'wb') as fh:
    warc_writer = WARCWriter(fh)
    with capture_http(warc_writer):
        requests.get('https://baidu.com/')

In [17]:
import pycountry

# print(len(pycountry.countries))

country_dict = dict()
for country in pycountry.countries:
    print(country.alpha_2,country.name)


AW Aruba
AF Afghanistan
AO Angola
AI Anguilla
AX Åland Islands
AL Albania
AD Andorra
AE United Arab Emirates
AR Argentina
AM Armenia
AS American Samoa
AQ Antarctica
TF French Southern Territories
AG Antigua and Barbuda
AU Australia
AT Austria
AZ Azerbaijan
BI Burundi
BE Belgium
BJ Benin
BQ Bonaire, Sint Eustatius and Saba
BF Burkina Faso
BD Bangladesh
BG Bulgaria
BH Bahrain
BS Bahamas
BA Bosnia and Herzegovina
BL Saint Barthélemy
BY Belarus
BZ Belize
BM Bermuda
BO Bolivia, Plurinational State of
BR Brazil
BB Barbados
BN Brunei Darussalam
BT Bhutan
BV Bouvet Island
BW Botswana
CF Central African Republic
CA Canada
CC Cocos (Keeling) Islands
CH Switzerland
CL Chile
CN China
CI Côte d'Ivoire
CM Cameroon
CD Congo, The Democratic Republic of the
CG Congo
CK Cook Islands
CO Colombia
KM Comoros
CV Cabo Verde
CR Costa Rica
CU Cuba
CW Curaçao
CX Christmas Island
KY Cayman Islands
CY Cyprus
CZ Czechia
DE Germany
DJ Djibouti
DM Dominica
DK Denmark
DO Dominican Republic
DZ Algeria
EC Ecuador
EG Eg